In [1]:
# Import the requests library.
import requests
import numpy as np
import pandas as pd
# Import the requests library.
import requests
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
# Dependencies for the wine API
import urllib
import json
# Import the API key.
from config import weather_api_key
import calendar

Token = 'tylRGHbxqiZUDNymMowCBFCuEqeULiWk'
Token_NOAA=Token
API_Token='Token d8f94b93417333267ca0edcca7dc14a9e035e1bb'

In [2]:
# https://github.com/crvaden/NOAA_API_v2
# https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859
# https://cran.r-project.org/web/packages/rnoaa/rnoaa.pdf
# file:///C:/Users/15124/Downloads/GHCND_documentation.pdf
# https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt

# California 1992 Prcp and Temp

In [86]:
# Create empty lists to append the data into
dates_temp = []
temps = []
dates_prcp = []
prcp = []

#Identify what years we want Temp and Prcp data for
for year in range(1992, 1993):
    year = str(year)
    #Print out 'working on year' to idenfity if script is running correctly
    print('working on year '+year)
    
    #make the api call for temp and precipitation
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USR0000CSRS&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
#     r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USR0000CSRS&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
    p = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USC00046074&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
    #Load JSON data
    d = json.loads(r.text)
    e = json.loads(p.text)
    #Get TAVG and PRCP data
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    avg_prcp = [item for item in e['results'] if item['datatype']=='PRCP']
    #get the date field from all average temperature readings
    dates_temp += [item['date'] for item in avg_temps]
    dates_prcp += [item['date'] for item in avg_prcp]
    #get the actual average temperature from all average temperature readings
    temps += [item['value'] for item in avg_temps]
    prcp += [item['value'] for item in avg_prcp]
    
# initialize dataframe
df_temp = pd.DataFrame()
df_prcp = pd.DataFrame()

# populate date and average temperature fields (cast string date to datetime)
df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
df_prcp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp]
# convert to degrees F
df_temp['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps]
#convert to mm to inches
df_prcp['avgPrcp'] = [float(v)*0.0393701 for v in prcp]
# Average the temp/prcp per day to temp/prcp per month
df_temp = df_temp.set_index('date').resample('M').mean()
df_prcp = df_prcp.set_index('date').resample('M').mean()
# Combine the series into a df
df_temp['avgPrcp']=df_prcp['avgPrcp']
df_temp.head(2)

working on year 1992


,avgTemp,avgPrcp
date,,
1992-01-31,44.530323,0.734062
1992-02-29,53.482069,2.530547


# Trying to get Washington 1992

In [119]:
#Create average temperature
dates_temp_MAX= []
dates_temp_MIN= []
temps_MAX = []
temps_MIN = []

#for each year from 2015-2019 ...
for year in range(1992, 2015):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:97218&datatypeid=TMAX&datatypeid=TMIN&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    
# #     print(d)
#     #get all items in the response which are average temperature readings
    avg_temps_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
    avg_temps_MIN = [item for item in d['results'] if item['datatype']=='TMIN']
    #get the date field from all average temperature readings
    dates_temp_MAX += [item['date'] for item in avg_temps_MAX]
    dates_temp_MIN += [item['date'] for item in avg_temps_MIN]
    #get the actual average temperature from all average temperature readings
    temps_MAX += [item['value'] for item in avg_temps_MAX]
    temps_MIN += [item['value'] for item in avg_temps_MIN]
    

# #initialize dataframe
df_temp_MAX = pd.DataFrame()
df_temp_MIN = pd.DataFrame()
df_temp_AVG = pd.DataFrame()


# #populate date and average temperature fields (cast string date to datetime
df_temp_MAX['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_MAX]
df_temp_MIN['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_MIN]
# #convert to degrees F
df_temp_MAX['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_MAX]
df_temp_MIN['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_MIN]

df_temp_MAX = df_temp_MAX.set_index('date').resample('M').mean()
df_temp_MIN = df_temp_MIN.set_index('date').resample('M').mean()
df_temp_MAX

working on year 1992
working on year 1993
working on year 1994
working on year 1995
working on year 1996
working on year 1997
working on year 1998
working on year 1999
working on year 2000
working on year 2001
working on year 2002
working on year 2003
working on year 2004
working on year 2005
working on year 2006
working on year 2007
working on year 2008
working on year 2009
working on year 2010
working on year 2011
working on year 2012
working on year 2013
working on year 2014


,avgTemp
date,
1992-01-31,41.087097
1992-02-29,42.117241
1992-03-31,43.473548
1992-04-30,43.574000
1992-05-31,45.790323
...,...
2014-08-31,47.259355
2014-09-30,46.172000
2014-10-31,44.245806


In [124]:
#Create average temperature
dates_temp_MAX= []
dates_temp_MIN= []
temps_MAX = []
temps_MIN = []

#for each year from 2015-2019 ...
for year in range(1992, 2015):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:99336&datatypeid=PRCP&datatypeid=TMIN&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    
# #     print(d)
#     #get all items in the response which are average temperature readings
    avg_temps_MAX = [item for item in d['results'] if item['datatype']=='PRCP']
    avg_temps_MIN = [item for item in d['results'] if item['datatype']=='TMIN']
    #get the date field from all average temperature readings
    dates_temp_MAX += [item['date'] for item in avg_temps_MAX]
    dates_temp_MIN += [item['date'] for item in avg_temps_MIN]
    #get the actual average temperature from all average temperature readings
    temps_MAX += [item['value'] for item in avg_temps_MAX]
    temps_MIN += [item['value'] for item in avg_temps_MIN]
    

# #initialize dataframe
df_temp_MAX = pd.DataFrame()
df_temp_MIN = pd.DataFrame()
df_temp_AVG = pd.DataFrame()


# #populate date and average temperature fields (cast string date to datetime
df_temp_MAX['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_MAX]
df_temp_MIN['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_MIN]
# #convert to degrees F
df_temp_MAX['avgTemp'] = temps_MAX
df_temp_MIN['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_MIN]

df_temp_MAX = df_temp_MAX.set_index('date').resample('M').mean()
df_temp_MIN = df_temp_MIN.set_index('date').resample('M').mean()
df_temp_MAX

working on year 1992
working on year 1993
working on year 1994
working on year 1995
working on year 1996
working on year 1997
working on year 1998
working on year 1999
working on year 2000
working on year 2001
working on year 2002
working on year 2003
working on year 2004
working on year 2005
working on year 2006
working on year 2007
working on year 2008
working on year 2009
working on year 2010
working on year 2011
working on year 2012
working on year 2013
working on year 2014


,avgTemp
date,
1992-01-31,0.023226
1992-02-29,0.038276
1992-03-31,0.007097
1992-04-30,0.044667
1992-05-31,0.000968
...,...
2014-07-31,0.000968
2014-08-31,0.008871
2014-09-30,0.000833


In [123]:
df_temp_MAX

,avgTemp
date,
1992-01-31,32.004181
1992-02-29,32.006890
1992-03-31,32.001277
1992-04-30,32.008040
1992-05-31,32.000174
...,...
2014-07-31,32.000174
2014-08-31,32.001597
2014-09-30,32.000150


# Trying to get for loop with multiple stations working

In [37]:
### initialize lists to store data
dates_prcp = []
prcp = []
zip_codes=['USW00023129', 'USW00023129']
temp_names=['avgPrcp', 'avgPrcp']
temp=['Napa_temp', 'Napa_temp']
bigdf = pd.DataFrame()
df_prcp = pd.DataFrame()

#for each year from 2015-2019 ...
for i in range(len(zip_codes)):
    zip_=zip_codes[i]
    for year, temps,t in zip(range(2013, 2014), temp_names, temp):
        year = str(year)
#         print('working on year '+year)

        #make the api call
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:{zip_}&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
        #load the api response as a json
        d = json.loads(r.text)
        #print(d)
        
        #json data with columns: date, datatype:prcp, stationID, attributes, value for each day
        #Data that will be used in prcp and date columns
        avg_prcp = [item for item in d['results'] if item['datatype']=='PRCP']
        #print(avg_prcp)
        
        #json data----- date column
        dates_prcp += [item['date'] for item in avg_prcp]
        #print(dates_prcp)
        
        #json data ----avgPrcp column
        prcp += [item['value'] for item in avg_prcp]
        #print(prcp)
        
        #create date column in DataFrame
        df_prcp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp]
        #print(df_prcp)
        
        #Empyt value column---create prcp column and convert values
        df_prcp[t] = [float(v)*0.0393701 for v in prcp]
        #print(df_prcp['avgPrcp'])
        
        #place the values of prcp in column
        df_prcp = df_prcp.set_index('date').resample('M').mean()
        #print(df_prcp)
        
        #df_prcp.rename(columns={temps:t}, inplace=True)
        df_prcp.reset_index(inplace=True)
        d=df_prcp[['date', t]]
        d['Date']=pd.to_datetime(d['date'], errors='coerce')
        bigdf['Date']=d['Date']
        bigdf[t] = d[t]
        print(bigdf)

bigdf

         Date  Napa_temp
0  2013-01-31   0.337821
1  2013-02-28   0.106862
2  2013-03-31   0.274321
3  2013-04-30   0.007874
4  2013-05-31   0.213361
5  2013-06-30   0.000000
6  2013-07-31   0.022860
7  2013-08-31   0.000000
8  2013-09-30   0.000000
9  2013-10-31   0.039370
10 2013-11-30   0.261155
11 2013-12-31   0.109220


ValueError: Length of values does not match length of index

In [2]:
#Create average temperature
dates_temp_MAX= []
dates_temp_MIN= []
temps_MAX = []
temps_MIN = []

# for i in range(len(zip_codes)):
#     zip_=zip_codes[i]
#for each year from 2015-2019 ...
for year in range(1992, 1993):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:99336&datatypeid=PRCP&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    
# #     print(d)
#     #get all items in the response which are average temperature readings
    avg_temps_MAX = [item for item in d['results'] if item['datatype']=='PRCP']
    #get the date field from all average temperature readings
    dates_temp_MAX += [item['date'] for item in avg_temps_MAX]
    #get the actual average temperature from all average temperature readings
    temps_MAX += [item['value'] for item in avg_temps_MAX]

working on year 1992


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
        #get all items in the response which are average temperature readings
        avg_temps_MAX = [item for item in d['results'] if item['datatype']=='PRCP']
        #get the date field from all average temperature readings
        dates_temp_MAX += [item['date'] for item in avg_temps_MAX]
        #get the actual average temperature from all average temperature readings
        temps_MAX += [item['value'] for item in avg_temps_MAX]

In [ ]:
# #initialize dataframe
df_temp_MAX = pd.DataFrame()

# #populate date and average temperature fields (cast string date to datetime
df_temp_MAX['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_MAX]
# #convert to degrees F
df_temp_MAX['avgTemp'] = temps_MAX

df_temp_MAX = df_temp_MAX.set_index('date').resample('M').mean()
df_temp_MAX

In [ ]:
#Create average temperature
dates_temp_W = []
temps_W = []
dates_prcp_W = []
prcp_W = []
names=['Napa', 'Walla', 'Salem']
zip_codes=['USW00023129', 'USW00023129', 'USW00023129']

#for each year from 2015-2019 ...
for i in range(len(zip_codes)):
    zip_=zip_codes[i]
    for year in range(2013, 2014):
        for name in names:
            year = str(year)
            print('working on year '+year)

            #make the api call for temp and precipitation
            r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:{zip_}&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
            p = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:{zip_}&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
            #Load JSON data
            d = json.loads(r.text)
            e = json.loads(p.text)
    #         print(d)
            #Get TAVG and PRCP data
            avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
            avg_prcp = [item for item in e['results'] if item['datatype']=='PRCP']
            #get the date field from all average temperature readings
            dates_temp_W += [item['date'] for item in avg_temps]
            dates_prcp_W += [item['date'] for item in avg_prcp]
            #get the actual average temperature from all average temperature readings
            temps_W += [item['value'] for item in avg_temps]
            prcp_W  += [item['value'] for item in avg_prcp]

            #initialize dataframe
            df_temp_W = pd.DataFrame()
            df_prcp_W = pd.DataFrame()

            #populate date and average temperature fields (cast string date to datetime
            df_temp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_W]
            df_prcp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_W]
            #convert to degrees F
            df_temp_W['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_W]
            #convert to mm to inches
            df_prcp_W['avgPrcp'] = [float(v)*0.0393701 for v in prcp_W]
            # Average the temp/prcp per day to temp/prcp per month
            df_temp_W = df_temp_W.set_index('date').resample('M').mean()
            df_prcp_W = df_prcp_W.set_index('date').resample('M').mean()
            # Combine the series into a df
            df_temp_W['avgPrcp']=df_prcp_W['avgPrcp']
#             df_temp_W.rename(columns={'date',name}, inplace=True)
#             d=df_temp_W[['date', name]]
#             d['Date']=pd.to_datetime(d['Date'], errors='coerce')
#             bigdf['Date']=d['Date']
#             bigdf[name] = d[name]
df_temp_W

# Trying to get zip codes working

In [ ]:
#Create average temperature
dates_temp = []
temps = []
# zip_codes=['USW00023129','USC00046074', 'USW00023129']
# zip_codes=['USW00023129']
zip_codes=['78735']


#for each year from 2015-2019 ...
# for i in zip_codes:
#     zip_=i
for year in range(2013, 2015):
    year = str(year)
    print('working on year '+year)

    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&locationid=ZIP:77040&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
#         print(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&locations?locationid=ZIP:77040&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token}')
    #load the api response as a json
    d = json.loads(r.text)
#     print(d)
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    #get the date field from all average temperature readings
    dates_temp += [item['date'] for item in avg_temps]
    #get the actual average temperature from all average temperature readings
    temps += [item['value'] for item in avg_temps]

#initialize dataframe
df_temp = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime
# df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
df_temp['date']=dates_temp
#convert to degrees F
df_temp['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps]
# df_temp = df_temp.set_index('date').resample('M').mean()
df_temp

In [ ]:
#Create average temperature
dates_temp_W = []
temps_W = []
dates_prcp_W = []
prcp_W = []
names=['Napa', 'Walla', 'Salem']
zip_codes=['USW00023129', 'USW00023129', 'USW00023129']
df_temp_W = pd.DataFrame()
temp_names=['avgTemp', 'avgTemp','avgTemp']
temp=['Napa_temp','Napa_temp','Napa_temp',]
bigdf = pd.DataFrame()

for year, temps, t, in zip(range(2013, 2014),temp_names, temp):
    for name in names:
        year = str(year)
        print('working on year '+year)

        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:{zip_}&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
        #Load JSON data
        d = json.loads(r.text)
        #print(d)
        #Get TAVG and PRCP data
        avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
        #get the date field from all average temperature readings
        dates_temp_W += [item['date'] for item in avg_temps]
        #get the actual average temperature from all average temperature readings
        temps_W += [item['value'] for item in avg_temps]

        #initialize dataframe
        df_temp_W = pd.DataFrame()

        #populate date and average temperature fields (cast string date to datetime
        df_temp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_W]
        #convert to degrees F
        df_temp_W['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_W]
        #convert to mm to inches
        # Average the temp/prcp per day to temp/prcp per month
        df_temp_W = df_temp_W.set_index('date').resample('M').mean()
        # Combine the series into a df
        df_temp_W['avgPrcp']=df_prcp_W['avgPrcp']          
        df_temp_W.rename(columns={temps:t}, inplace=True)
        df_temp_W.reset_index(inplace=True)
        d=df_temp_W[['date', t]]
        bigdf['Date']=d['date']
        bigdf[t] = d[t]
df_temp_W 

# Trying to get zip code - Not working

In [ ]:
### initialize lists to store data
dates_prcp = []
prcp = []
zip_codes=['78735']
#for each year from 2015-2019 ...
for i in range(len(zip_codes)):
    zip_=zip_codes[i]
    for year in range(1992, 2017):
        year = str(year)
    #     print('working on year '+year)

        #make the api call
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&locations?locationid=ZIP:{zip_}&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
        #load the api response as a json
        d = json.loads(r.text)
        #get all items in the response which are Precipitation temperature readings
        avg_prcp = [item for item in d['results'] if item['datatype']=='PRCP']
        #get the date field from all average Precipitation readings
        dates_prcp += [item['date'] for item in avg_prcp]
        #get the actual average Precipitation from all average Precipitation readings
        prcp += [item['value'] for item in avg_prcp]

#initialize dataframe
df_prcp = pd.DataFrame()

#populate date and average Precipitation fields (cast string date to datetime
df_prcp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp]
#convert to mm to inches
df_prcp['avgPrcp'] = [float(v)*0.0393701 for v in prcp]
df_prcp = df_prcp.set_index('date').resample('M').mean()

# Working one station, only temperature

In [ ]:
#Create average temperature
dates_temp_W = []
temps_W = []

#for each year from 2015-2019 ...
for year in range(2013, 2017):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    #get the date field from all average temperature readings
    dates_temp_W += [item['date'] for item in avg_temps]
    #get the actual average temperature from all average temperature readings
    temps_W += [item['value'] for item in avg_temps]

#initialize dataframe
df_temp_W = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime
df_temp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_W]
#convert to degrees F
df_temp_W['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_W]
df_temp_W = df_temp_W.set_index('date').resample('M').mean()
df_temp_W

# Working one station, only prcp 1992

In [38]:
### initialize lists to store data
dates_prcp_W = []
prcp_W = []

#for each year from 2015-2019 ...
for year in range(1992, 2017):
    year = str(year)
#     print('working on year '+year)
    
    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USC00046074&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are Precipitation temperature readings
    avg_prcp = [item for item in d['results'] if item['datatype']=='PRCP']
    #get the date field from all average Precipitation readings
    dates_prcp_W += [item['date'] for item in avg_prcp]
    #get the actual average Precipitation from all average Precipitation readings
    prcp_W += [item['value'] for item in avg_prcp]

#initialize dataframe
df_prcp_W = pd.DataFrame()

#populate date and average Precipitation fields (cast string date to datetime
df_prcp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_W]
#convert to mm to inches
df_prcp_W['avgPrcp'] = [float(v)*0.0393701 for v in prcp_W]
df_prcp_W = df_prcp_W.set_index('date').resample('M').mean()
df_prcp_W

,avgPrcp
date,
1992-01-31,0.734062
1992-02-29,2.530547
1992-03-31,1.381764
1992-04-30,0.209974
1992-05-31,0.000000
...,...
2016-08-31,0.000000
2016-09-30,0.000000
2016-10-31,1.285153


In [12]:
### initialize lists to store data
dates_prcp_W = []
prcp_W = []

#for each year from 2015-2019 ...
for year in range(1992, 2017):
    year = str(year)
#     print('working on year '+year)
    
    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are Precipitation temperature readings
    avg_prcp = [item for item in d['results'] if item['datatype']=='PRCP']
    #get the date field from all average Precipitation readings
    dates_prcp_W += [item['date'] for item in avg_prcp]
    #get the actual average Precipitation from all average Precipitation readings
    prcp_W += [item['value'] for item in avg_prcp]

#initialize dataframe
df_prcp_W = pd.DataFrame()

#populate date and average Precipitation fields (cast string date to datetime
df_prcp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_W]
#convert to mm to inches
df_prcp_W['avgPrcp'] = [float(v)*0.0393701 for v in prcp_W]
df_prcp_W = df_prcp_W.set_index('date').resample('M').mean()
df_prcp_W

,avgPrcp
date,
1992-01-31,0.477521
1992-02-29,1.565301
1992-03-31,1.706884
1992-04-30,0.006562
1992-05-31,0.006350
...,...
2016-08-31,0.000000
2016-09-30,0.000000
2016-10-31,0.133350


# Trying to create one DF for a for loop of multiple stations

In [ ]:
#Create average temperature
dates_temp_W = []
temps_W = []
dates_prcp_W = []
prcp_W = []
names=['Napa', 'Walla', 'Salem']
zip_codes=['USW00023129', 'USW00023129', 'USW00023129']

df_temp_W = pd.DataFrame()
df_prcp_W = pd.DataFrame()


temp_names=['avgTemp', 'avgTemp','avgTemp']
temp=['Napa_temp','Napa_temp','Napa_temp',]
prcp_names=['avgPrcp', 'avgPrcp', 'avgPrcp']
prcp=['Napa_prcp', 'Napa_prcp', 'Napa_prcp']
bigdf = pd.DataFrame()


#for each year from 2015-2019 ...
for i in range(len(zip_codes)):
    zip_=zip_codes[i]
    for year, temps, t, prcps, p in zip(range(2013, 2014),temp_names, temp, prcp_names, prcp):
        for name in names:
            year = str(year)
            print('working on year '+year)

            #make the api call for temp and precipitation
            r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:{zip_}&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
            p = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:{zip_}&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
            #Load JSON data
            d = json.loads(r.text)
            e = json.loads(p.text)
    #         print(d)
            #Get TAVG and PRCP data
            avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
            avg_prcp = [item for item in e['results'] if item['datatype']=='PRCP']
            #get the date field from all average temperature readings
            dates_temp_W += [item['date'] for item in avg_temps]
            dates_prcp_W += [item['date'] for item in avg_prcp]
            #get the actual average temperature from all average temperature readings
            temps_W += [item['value'] for item in avg_temps]
            prcp_W  += [item['value'] for item in avg_prcp]

            #initialize dataframe
            df_temp_W = pd.DataFrame()
            df_prcp_W = pd.DataFrame()

            #populate date and average temperature fields (cast string date to datetime
            df_temp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_W]
            df_prcp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_W]
            #convert to degrees F
            df_temp_W['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_W]
            #convert to mm to inches
            df_prcp_W['avgPrcp'] = [float(v)*0.0393701 for v in prcp_W]
            # Average the temp/prcp per day to temp/prcp per month
            df_temp_W = df_temp_W.set_index('date').resample('M').mean()
            df_prcp_W = df_prcp_W.set_index('date').resample('M').mean()
            # Combine the series into a df
            df_temp_W['avgPrcp']=df_prcp_W['avgPrcp']          
#             df_temp_W.rename(columns={temps:t}, inplace=True)
#             df_temp_W.rename(columns={prcps:p}, inplace=True)
#             df_temp_W.reset_index(inplace=True)
#             d=df_temp_W[['date', t, p]]
#             bigdf['Date']=d['date']
#             bigdf[t] = d[t]
#             bigdf[p] = d[p]
df_temp_W   

In [ ]:
temp_names=['avgTemp']
temp=['Napa_temp', 'walla']
prcp_names=['avgPrcp']
prcp=['Napa_prcp']
bigdf = pd.DataFrame()


for temps, t, prcps,p in zip(temp_names, temp, prcp_names, prcp):
    df_temp_W.rename(columns={temps:t}, inplace=True)
    df_temp_W.rename(columns={prcps:p}, inplace=True)
    df_temp_W.reset_index(inplace=True)
#     d=df_temp_W[['date', t, p]]
    bigdf['Date']=df_temp_W['date']
    bigdf[t] = df_temp_W[t]
    bigdf[p] = df_temp_W[p]
bigdf
#             d['Date']=pd.to_datetime(d['Date'], errors='coerce')
#             bigdf['Date']=d['Date']
#             bigdf[name] = d[name]

In [ ]:
df_temp_W['avgPrcp']=df_prcp_W['avgPrcp']
df_temp_W.to_csv("Washington_Weather.csv")

#  Oregon Avg Temp and Precipitation

In [ ]:
#Create average temperature
dates_temp_O = []
temps_O = []

#for each year from 2015-2019 ...
for year in range(2013, 2017):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    #get the date field from all average temperature readings
    dates_temp_O += [item['date'] for item in avg_temps]
    #get the actual average temperature from all average temperature readings
    temps_O += [item['value'] for item in avg_temps]

#initialize dataframe
df_temp_O = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime
df_temp_O['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_O]
#convert to degrees F
df_temp_O['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_O]
df_temp_O = df_temp_O.set_index('date').resample('M').mean()

In [ ]:
### initialize lists to store data
dates_prcp_O = []
prcp_O = []

#for each year from 2015-2019 ...
for year in range(1992, 2017):
    year = str(year)
#     print('working on year '+year)
    
    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USC00046074&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are Precipitation temperature readings
    avg_prcp = [item for item in d['results'] if item['datatype']=='PRCP']
    #get the date field from all average Precipitation readings
    dates_prcp_O += [item['date'] for item in avg_prcp]
    #get the actual average Precipitation from all average Precipitation readings
    prcp_O += [item['value'] for item in avg_prcp]

#initialize dataframe
df_prcp_O = pd.DataFrame()

#populate date and average Precipitation fields (cast string date to datetime
df_prcp_O['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_O]
#convert to mm to inches
df_prcp_O['avgPrcp'] = [float(v)*0.0393701 for v in prcp_O]
df_prcp_O = df_prcp_O.set_index('date').resample('M').mean()

In [ ]:
df_temp_O['avgPrcp']=df_prcp_O['avgPrcp']
df_temp_O.to_csv("Oregon_Weather.csv")

# Add Lat/Long/Elevation

In [ ]:
# create new columns for Lat/Long/Elevation

#California_GHCND:
Nappa_Valley_g='GHCND:USC00046074'
# Washington_GHCND:
Walla_Walla_Valley_g='GHCND:GHCND:USC00457015'
# Oregon_GHCND:
Willamette_Valley_g='GHCND:USW00024232'
#--------------------------------------------------------Latitude-----------------------------------------------------#

#Latitude
#California_LAT:
Nappa_Valley_lat='38.2777'
# Oregon_lat:
Willamette_Valley_lat='44.905'
# Washington_Lat:
Walla_Walla_Valley_lat='46.3119'


#Latitude Red
latitude=[]
for lat in red_USA_df['regions']:
#California
    if lat =='California':
        latitude.append('38.2777')
#Oregon
    elif lat == 'Oregon':
        latitude.append('44.905')
#Washington
    else:
        latitude.append('46.3119')
red_USA_df['lat']=latitude

#Latitude White
latitude=[]
for lat in white_USA_df['regions']:
#California
    if lat =='California':
        latitude.append('38.2777')
#Oregon
    elif lat == 'Oregon':
        latitude.append('44.905')
#Washington
    else:
        latitude.append('46.3119')
white_USA_df['lat']=latitude

#------------------------------------------------------Longitude-------------------------------------------------------------#

#Longitude
#California_Long:
Nappa_Valley_long='-122.2647'
# Oregon_Long:
Willamette_Valley_long='-123.0011'
# Washington_Long:
Walla_Walla_Valley_long= '-119.2633'

#Longitude Red
longitude=[]
for long in red_USA_df['regions']:
#California
    if long =='California':
        longitude.append('-122.2647')
#Oregon
    elif long == 'Oregon':
        longitude.append('-123.0011')
#Washington
    else:
        longitude.append('-119.2633')
red_USA_df['long']=longitude 

#Longitude White
longitude=[]
for long in white_USA_df['regions']:
#California
    if long =='California':
        longitude.append('-122.2647')
#Oregon
    elif long == 'Oregon':
        longitude.append('-123.0011')
#Washington
    else:
        longitude.append('-119.2633')
white_USA_df['long']=longitude 

#-----------------------------------------------------------------Elevation------------------------------------------------#

#Elevation (feet)
#California_Elevation:
Nappa_Valley_long='35.1'
# Oregon_Elevation:
Willamette_Valley_long='205.1'
# Washington_Elevation:
Walla_Walla_Valley_long= '631.9'


#Elevation Red
elevation=[]
for ele in red_USA_df['regions']:
#California
    if ele =='California':
        elevation.append('35.1')
#Oregon
    elif ele == 'Oregon':
        elevation.append('205.1')
#Washington
    else:
        elevation.append('631.9')
red_USA_df['elevation']=elevation

#Elevation White
elevation=[]
for ele in white_USA_df['regions']:
#California
    if ele =='California':
        elevation.append('35.1')
#Oregon
    elif ele == 'Oregon':
        elevation.append('205.1')
#Washington
    else:
        elevation.append('631.9')
white_USA_df['elevation']=elevation


In [ ]:
white_USA_df.head(2)

# Add Temp/Precipitation

In [ ]:
# Split the date into month and year columns
#-------------------------------------------------------------------------------California----------------------------
df_temp.reset_index(inplace= True )
df_temp['day'] = df_temp['date'].dt.day
df_temp['month'] = df_temp['date'].dt.month
df_temp['year'] = df_temp['date'].dt.year

list_months=[]
for x in df_temp['month']:
    list_months.append(calendar.month_name[x])
df_temp['months']=list_months
df_temp.drop(['date', 'day','month'], axis=1, inplace=True)
#----------------------------------------------------------------------------------Oregon-------------------------------
df_temp_O.reset_index(inplace= True )
df_temp_O['day'] = df_temp_O['date'].dt.day
df_temp_O['month'] = df_temp_O['date'].dt.month
df_temp_O['year'] = df_temp_O['date'].dt.year

list_months=[]
for x in df_temp_O['month']:
    list_months.append(calendar.month_name[x])
df_temp_O['months']=list_months
df_temp_O.drop(['date', 'day','month'], axis=1, inplace=True)
#-----------------------------------------------------------------------------------Washington---------------------
df_temp_W.reset_index(inplace= True )
df_temp_W['day'] = df_temp_W['date'].dt.day
df_temp_W['month'] = df_temp_W['date'].dt.month
df_temp_W['year'] = df_temp_W['date'].dt.year

list_months=[]
for x in df_temp_W['month']:
    list_months.append(calendar.month_name[x])
df_temp_W['months']=list_months
df_temp_W.drop(['date', 'day','month'], axis=1, inplace=True)

In [ ]:
df_temp.tail(2)

In [ ]:
white_USA_df.head(2)

In [ ]:
list_temp_march=[]
for u,v,x,y,z in zip(white_USA_df['regions'],white_USA_df['vintage'],df_temp['year'], df_temp['avgTemp'], df_temp['months']):
    if v==x and v==2016 and u=='California'and z=='March':
        list_temp_march.append(y)
    elif y==z and y==2016 and x=='Oregon'and v=='March':
        list_temp_march.append(u)
    else:
        list_temp_march.append(u)
white_USA_df['march']=list_temp_march
white_USA_df
        
        
        

In [ ]:
# sorted_white_USA_df=white_USA_df.sort_values(by=['regions'], ascending=False)
# sorted_white_USA_df.head(50)